# RAG application built on gemini 

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("yolov9_paper.pdf")
data = loader.load()  # entire PDF is loaded as a single Document
#data

In [3]:
len(data)

18

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)


print("Total number of documents: ",len(docs))

Total number of documents:  96


In [5]:
docs[7]

Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-03-01T01:40:26+00:00', 'author': '', 'keywords': '', 'moddate': '2024-03-01T01:40:26+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'yolov9_paper.pdf', 'total_pages': 18, 'page': 1, 'page_label': '2'}, page_content='ditional layers to combine repeatedly fed input data, which\nwill significantly increase the inference cost. In addition,\nsince the input data layer to the output layer cannot have a\ntoo deep path, this limitation will make it difficult to model\nhigh-order semantic information during the training pro-\ncess. As for masked modeling, its reconstruction loss some-\ntimes conflicts with the target loss. In addition, most mask\nmechanisms also produce incorrect associations with data.\nFor the deep supervision mechanism, it will produce error\nac

In [6]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from dotenv import load_dotenv
load_dotenv() 

#Get an API key: 
# Head to https://ai.google.dev/gemini-api/docs/api-key to generate a Google AI API key. Paste in .env file

# Embedding models: https://python.langchain.com/v0.1/docs/integrations/text_embedding/

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]
#vector

/opt/anaconda3/envs/first/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734263241291,
 0.01813093200325966]

In [7]:
vectorstore = Chroma.from_documents(documents=docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [8]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

retrieved_docs = retriever.invoke("What is new in yolov9?")


In [9]:
len(retrieved_docs)

10

In [10]:
print(retrieved_docs[5].page_content)

target task, it also overcomes the problems encountered by
mask modeling. The proposed PGI mechanism can be ap-
plied to deep neural networks of various sizes and is more
general than the deep supervision mechanism, which is only
suitable for very deep neural networks.
In this paper, we also designed generalized ELAN
(GELAN) based on ELAN [65], the design of GELAN si-
multaneously takes into account the number of parameters,
computational complexity, accuracy and inference speed.
This design allows users to arbitrarily choose appropriate
computational blocks for different inference devices. We
combined the proposed PGI and GELAN, and then de-
signed a new generation of YOLO series object detection
system, which we call YOLOv9. We used the MS COCO
dataset to conduct experiments, and the experimental results
verified that our proposed YOLOv9 achieved the top perfor-
mance in all comparisons.
We summarize the contributions of this paper as follows:


In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.3, max_tokens=500)

In [12]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [13]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [14]:
response = rag_chain.invoke({"input": "what is new in YOLOv9?"})
print(response["answer"])

YOLOv9 introduces Programmable Gradient Information (PGI) to improve information flow and gradient flow during training.  It also features Generalized ELAN (GELAN) for improved architecture, balancing parameters, computation, accuracy, and inference speed.  These improvements make YOLOv9 a top-performing real-time object detector.
